In [25]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
from ipywidgets import interact, interactive, fixed
import ipywidgets
from IPython.display import display, HTML, Image, clear_output

In [14]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [15]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT'""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [34]:
rad_data = rad_data[rad_data.text.str.contains("chest")]
print(len(rad_data))

20552


In [35]:
def view_reports(anno_docs):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(anno_docs)-1))
    def _view_markup(i):
        clear_output()
        #report_html = pneumonia_annotation_html_markup(anno_docs[i])
        #report_html = report_html.replace('\n', '<br>')
        print(anno_docs.iloc[i]["text"])

In [36]:
view_reports(rad_data)

In [14]:
imp_reports = [row["text"] for _, row in rad_data.iterrows() if "IMPRESSION:" in row['text'] and 
                                                                "COMPARISON:" in row["text"] and
                                                                "TECHNIQUE:" in row['text'] and 
                                                                "FINDINGS:" in row['text'] and
                                                                "HISTORY:" in row["text"]]

In [15]:
print(imp_reports[1])




     DATE: [**3352-12-14**] 12:53 PM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 1656**]
     Reason: please evaluate brain, midbrain, brainstem for cn lesions es
     Admitting Diagnosis: NON-HODGKIN'S LYMPHOMA;NEUTROPENIC;FEVER
      Contrast: MAGNEVIST Amt: 17
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      55 year old man with h/o EATL s/p auto bmt with recent right facial droop and 
      new double vision right [**Hospital Unit Name 626**]
     REASON FOR THIS EXAMINATION:
      please evaluate brain, midbrain, brainstem for cn lesions especially 6          
     ______________________________________________________________________________
                                     FINAL REPORT
     HISTORY:  55-year-old man with history of T-cell lymphoma with recent right
     facial droop and new right-sided double vision.
     
     COMPARISON:

### Write a function that returns the impression section of a report

#### Hints

* Not every report will have an impression section
* "INTERPRETATION" and "CONCLUSIONS" might be synonyms for "IMPRESSION"

### Define Regular expressions for data cleansing

* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [ ]:
for i in range(10):
    print(rad_data.iloc[i]['text'])
    print("*"*42,"\n\n")

### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

### Write a function to drop [stop words](https://en.wikipedia.org/wiki/Stop_words) from a report

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.